In [325]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score


In [326]:
df=pd.read_csv('/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/all_matches.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [327]:
df.season.unique()

array(['2007/08', '2009', '2009/10', '2011', '2012', 2012, 2013, 2014,
       2015, 2016, 2017, 2019, 2018, '2018', '2019', '2020/21', '2021'],
      dtype=object)

In [328]:
season_map = {'2007/08':2008, '2009':2009, '2009/10':2010, '2011':2011, '2012':2012, 2012:2012, 2013:2013, 2014:2014,
       2015:2015, 2016:2016, 2017:2017, 2019:2019, 2018:2018, '2018':2018, '2019':2019, '2020/21':2020, '2021':2021}

In [329]:
df.season = df.season.map(season_map)

In [330]:
df.season.unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2019,
       2018, 2020, 2021])

In [331]:
def cleaner(x):
  a= " ".join(x.replace('.', ' ').split())
  b= "".join(a.split(',')[0])    
  return b

df['venue']=df['venue'].apply(cleaner)


In [332]:
def converter(x):
  if x=='Feroz Shah Kotla':
    x='Arun Jaitley Stadium'
    return x
  else:
    return x
 
df['venue']=df['venue'].apply(converter) 
df['venue'].unique()

array(['M Chinnaswamy Stadium', 'Punjab Cricket Association Stadium',
       'Arun Jaitley Stadium', 'Eden Gardens', 'Wankhede Stadium',
       'Sawai Mansingh Stadium', 'Rajiv Gandhi International Stadium',
       'MA Chidambaram Stadium', 'Dr DY Patil Sports Academy', 'Newlands',
       "St George's Park", 'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium', 'Sardar Patel Stadium',
       'Barabati Stadium', 'Vidarbha Cricket Association Stadium',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr Y S Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
       'Maharashtra Cricket Association Stad

In [333]:
df['total_runs']=df.runs_off_bat + df.extras

In [334]:
df['total_runs_cum'] = df.groupby(['match_id','innings'])['total_runs'].cumsum()

In [335]:
df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,total_runs,total_runs_cum
0,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,1
1,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
3,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
4,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194349,1254060,2021,2021-04-11,MA Chidambaram Stadium,2,19.2,Sunrisers Hyderabad,Kolkata Knight Riders,Abdul Samad,MK Pandey,AD Russell,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,168
194350,1254060,2021,2021-04-11,MA Chidambaram Stadium,2,19.3,Sunrisers Hyderabad,Kolkata Knight Riders,Abdul Samad,MK Pandey,AD Russell,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,169
194351,1254060,2021,2021-04-11,MA Chidambaram Stadium,2,19.4,Sunrisers Hyderabad,Kolkata Knight Riders,MK Pandey,Abdul Samad,AD Russell,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,170
194352,1254060,2021,2021-04-11,MA Chidambaram Stadium,2,19.5,Sunrisers Hyderabad,Kolkata Knight Riders,Abdul Samad,MK Pandey,AD Russell,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,171


In [336]:
xyz = df.groupby(['season','venue','match_id','innings'],as_index=False).agg({'total_runs':'sum'})
xyz['whole_runs'] = xyz['total_runs']
xyz.drop(columns='total_runs',inplace=True)
xyz.to_csv('/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/table.csv')
xyz

,season,venue,match_id,innings,whole_runs
0,2008,Arun Jaitley Stadium,335984,1,129
1,2008,Arun Jaitley Stadium,335984,2,132
2,2008,Arun Jaitley Stadium,335998,1,191
3,2008,Arun Jaitley Stadium,335998,2,181
4,2008,Arun Jaitley Stadium,336009,1,187
...,...,...,...,...,...
1659,2021,MA Chidambaram Stadium,1254058,2,160
1660,2021,MA Chidambaram Stadium,1254060,1,187
1661,2021,MA Chidambaram Stadium,1254060,2,177
1662,2021,Wankhede Stadium,1254059,1,188


In [337]:
df2 = df.merge(xyz)

In [338]:
abc = df2.groupby(['venue','season','innings'],as_index=False).agg({'whole_runs':'mean'})
abc['whole_runs_per_venue_season_innings'] = abc['whole_runs']
abc.drop(columns='whole_runs',inplace=True)
abc.to_csv('/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/alyal_table.csv')
abc

,venue,season,innings,whole_runs_per_venue_season_innings
0,Arun Jaitley Stadium,2008,1,169.522302
1,Arun Jaitley Stadium,2008,2,167.856693
2,Arun Jaitley Stadium,2010,1,172.682759
3,Arun Jaitley Stadium,2010,2,138.271394
4,Arun Jaitley Stadium,2011,1,150.272160
...,...,...,...,...
291,Wankhede Stadium,2019,2,166.887694
292,Wankhede Stadium,2019,3,8.000000
293,Wankhede Stadium,2019,4,9.000000
294,Wankhede Stadium,2021,1,188.000000


In [339]:
df3 = df2.merge(abc)

In [340]:
df3

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,total_runs,total_runs_cum,whole_runs,whole_runs_per_venue_season_innings
0,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,1,222,156.580796
1,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,222,156.580796
2,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,222,156.580796
3,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,222,156.580796
4,335982,2008,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,222,156.580796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194349,1254059,2021,2021-04-10,Wankhede Stadium,2,17.6,Delhi Capitals,Chennai Super Kings,MP Stoinis,RR Pant,DJ Bravo,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,182,190,190.000000
194350,1254059,2021,2021-04-10,Wankhede Stadium,2,18.1,Delhi Capitals,Chennai Super Kings,MP Stoinis,RR Pant,SN Thakur,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,186,190,190.000000
194351,1254059,2021,2021-04-10,Wankhede Stadium,2,18.2,Delhi Capitals,Chennai Super Kings,MP Stoinis,RR Pant,SN Thakur,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,186,190,190.000000
194352,1254059,2021,2021-04-10,Wankhede Stadium,2,18.3,Delhi Capitals,Chennai Super Kings,MP Stoinis,RR Pant,SN Thakur,0,0,NaN,NaN,NaN,NaN,NaN,caught,MP Stoinis,NaN,NaN,0,186,190,190.000000


In [341]:
df3.to_csv('/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/final_table_when_alyal_came.csv')

In [342]:
df=df.drop(labels=['striker', 'non_striker', 'bowler', 'wides','noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','player_dismissed', 'other_wicket_type', 'other_player_dismissed' ,'total_runs','runs_off_bat','extras'],axis=1)

In [343]:
consistent_team=['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Mumbai Indians','Sunrisers Hyderabad','Delhi Capitals']

In [344]:
df = df[(df['batting_team'].isin(consistent_team)) & (df['bowling_team'].isin(consistent_team))]

In [345]:
#df =df[df['ball']<= 5.0]

In [346]:
df=df[df['innings']<=2]

In [347]:
df=df.drop(labels=['match_id'],axis=1)

In [348]:
df=df.drop(labels=['venue'],axis=1)

In [349]:
df

,season,start_date,innings,ball,batting_team,bowling_team,total_runs_cum
0,2008,2008-04-18,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,1
1,2008,2008-04-18,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,1
2,2008,2008-04-18,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,2
3,2008,2008-04-18,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,2
4,2008,2008-04-18,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,2
...,...,...,...,...,...,...,...
194349,2021,2021-04-11,2,19.2,Sunrisers Hyderabad,Kolkata Knight Riders,168
194350,2021,2021-04-11,2,19.3,Sunrisers Hyderabad,Kolkata Knight Riders,169
194351,2021,2021-04-11,2,19.4,Sunrisers Hyderabad,Kolkata Knight Riders,170
194352,2021,2021-04-11,2,19.5,Sunrisers Hyderabad,Kolkata Knight Riders,171


In [350]:
df.batting_team.unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Mumbai Indians',
       'Rajasthan Royals', 'Sunrisers Hyderabad', 'Delhi Capitals'],
      dtype=object)

In [351]:
team_encoder = LabelEncoder()
df['batting_team'] = team_encoder.fit_transform(df['batting_team'])
df['bowling_team'] = team_encoder.fit_transform(df['bowling_team'])

In [352]:
df

,season,start_date,innings,ball,batting_team,bowling_team,total_runs_cum
0,2008,2008-04-18,1,0.1,3,6,1
1,2008,2008-04-18,1,0.2,3,6,1
2,2008,2008-04-18,1,0.3,3,6,2
3,2008,2008-04-18,1,0.4,3,6,2
4,2008,2008-04-18,1,0.5,3,6,2
...,...,...,...,...,...,...,...
194349,2021,2021-04-11,2,19.2,7,3,168
194350,2021,2021-04-11,2,19.3,7,3,169
194351,2021,2021-04-11,2,19.4,7,3,170
194352,2021,2021-04-11,2,19.5,7,3,171


In [353]:
df['start_date'] = df['start_date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [354]:
X_train = df.drop(labels='total_runs_cum', axis=1)[df['start_date'].dt.year <= 2016]
X_test =  df.drop(labels='total_runs_cum', axis=1)[df['start_date'].dt.year >= 2017]

y_train =df[df['start_date'].dt.year <= 2016]['total_runs_cum'].values
y_test = df[df['start_date'].dt.year >= 2017]['total_runs_cum'].values

In [355]:
X_train.drop(labels='start_date',axis=1,inplace=True)
X_test.drop(labels='start_date',axis=1,inplace=True)

In [356]:
X_train

,season,innings,ball,batting_team,bowling_team
0,2008,1,0.1,3,6
1,2008,1,0.2,3,6
2,2008,1,0.3,3,6
3,2008,1,0.4,3,6
4,2008,1,0.5,3,6
...,...,...,...,...,...
136593,2016,2,19.2,6,7
136594,2016,2,19.3,6,7
136595,2016,2,19.4,6,7
136596,2016,2,19.5,6,7


In [357]:
#rfr=RandomForestRegressor()
#rfr.fit(X_train,y_train)
#pred=rfr.predict(X_test)
#print(r2_score(y_test,pred))

In [358]:
#joblib.dump(rfr,'/content/drive/MyDrive/Hackathon/IIT Madras/Permanent/regressor_model.joblib')
#joblib.dump(team_encoder,'/content/drive/MyDrive/Hackathon/IIT Madras/Permanent//team_encoder.joblib')